In [ ]:
# instaling the necessary drive
!python -m pip install pandas 

In [ ]:
!python -m pip install azure_storage_blob

In [7]:
!python -m pip install python-dotenv

In [8]:
#Importing Necessary libraries
import pandas as pd
from azure.storage.blob import BlobServiceClient
from dotenv import load_dotenv
import os

In [10]:
# Data Extraction
try:
    data = pd.read_csv(r'zipco_transaction.csv')
    print('Data Extracted Successfully!')
except Exception as e:
    print(f"An erro occurred: {e}")

Data Extracted Successfully!


In [11]:
data.head(3
          )

,Date,ProductName,Quantity,UnitPrice,StoreLocation,PaymentType,PromotionApplied,Weather,Temperature,StaffPerformanceRating,...,DeliveryTime_min,OrderType,CustomerName,CustomerAddress,Customer_PhoneNumber,CustomerEmail,Staff_Name,Staff_Email,DayOfWeek,TotalSales
0,2023-01-01 00:00:00,Vanilla Cake,2,12.532304,South,Cash,True,Rainy,20.654914,Poor,...,30,In Store,William Adams,"9851 David Green\r\nTonyaburgh, VA 02853",(916)427-7276x861,lisa00@example.net,John Bridges,pdavidson@example.com,Sunday,25.064608
1,2023-01-01 01:00:00,Red Velvet Cake,1,7.083070,South,Cash,False,Rainy,23.549497,Average,...,33,In Store,Anthony Wiggins,"24682 Holly Stravenue\r\nMooreville, NH 13901",769.318.4373,michellefernandez@example.com,Sarah Bentley,ajohnson@example.net,Sunday,7.083070
2,2023-01-01 02:00:00,Chocolate Cake,5,6.736064,North,Cash,True,Rainy,NaN,Excellent,...,43,Phone Order,Ashley Duke,10184 Washington Trace Apt. 679\r\nEast Brandi...,702.520.3286,cooperwilliam@example.com,Connie Cervantes,michele29@example.net,Sunday,33.680321


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005 entries, 0 to 1004
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Date                    1005 non-null   object 
 1   ProductName             1005 non-null   object 
 2   Quantity                1005 non-null   int64  
 3   UnitPrice               1005 non-null   float64
 4   StoreLocation           1005 non-null   object 
 5   PaymentType             1005 non-null   object 
 6   PromotionApplied        1005 non-null   bool   
 7   Weather                 1005 non-null   object 
 8   Temperature             904 non-null    float64
 9   StaffPerformanceRating  1005 non-null   object 
 10  CustomerFeedback        905 non-null    object 
 11  DeliveryTime_min        1005 non-null   int64  
 12  OrderType               1005 non-null   object 
 13  CustomerName            1005 non-null   object 
 14  CustomerAddress         1005 non-null   

In [13]:
# Data cleaning and transformation
# Remove duplicates
data.drop_duplicates(inplace=True)

In [16]:
# Hnadle missing values( filling missing numeric values with the mean or median)
numeric_columns = data.select_dtypes(include =['float', 'int64']).columns
for col in numeric_columns:
    data.fillna({col: data[col].mean()}, inplace =True)

In [18]:
# Handling missing values (filling missing string object values with 'unknown')
String_columns = data.select_dtypes(include =['object']).columns
for col in String_columns:
    data.fillna({col: 'Unknown'}, inplace=True)

In [36]:
data['Date'] = pd.to_datetime(data['Date'])

In [37]:
#Create products table
products= data [['ProductName' ]].copy().drop_duplicates().reset_index(drop=True)
products.index.name ='ProductID'
products = products.reset_index()

In [38]:
customers =data [['CustomerName','CustomerAddress','Customer_PhoneNumber','CustomerEmail']].copy().drop_duplicates().reset_index(drop=True)
customers.index.name ='CustomerID'
customers = customers.reset_index()#create customers table


In [39]:
# creating the staff
staff =data [['Staff_Name','Staff_Email']].copy().drop_duplicates().reset_index(drop=True)
staff.index.name ='StaffID'
staff = staff.reset_index()

In [40]:
# Creating the transactons tabel
transaction = data.merge(products, on=['ProductName'], how = 'left') \
                    .merge(customers, on = ['CustomerName','CustomerAddress','Customer_PhoneNumber','CustomerEmail'], how ='left' ) \
                    .merge(staff, on= ['Staff_Name','Staff_Email'], how ='left' )
transaction.index.name ='TransactionID'
transaction = transaction.reset_index()\
                            [['Date', 'ProductName', 'Quantity', 'UnitPrice', 'StoreLocation','ProductID', \
                              'PaymentType', 'PromotionApplied', 'Weather', 'Temperature','StaffPerformanceRating','CustomerID', \
                                'CustomerFeedback', 'DeliveryTime_min','OrderType','StaffID','DayOfWeek','TotalSales']]

In [41]:
transaction.info(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Date                    1000 non-null   datetime64[ns]
 1   ProductName             1000 non-null   object        
 2   Quantity                1000 non-null   int64         
 3   UnitPrice               1000 non-null   float64       
 4   StoreLocation           1000 non-null   object        
 5   ProductID               1000 non-null   int64         
 6   PaymentType             1000 non-null   object        
 7   PromotionApplied        1000 non-null   bool          
 8   Weather                 1000 non-null   object        
 9   Temperature             1000 non-null   float64       
 10  StaffPerformanceRating  1000 non-null   object        
 11  CustomerID              1000 non-null   int64         
 12  CustomerFeedback        1000 non-null   object   

In [42]:
# Save data to as csv files
data.to_csv('clean_data.csv', index=False)
products.to_csv('products.csv', index=False)
customers.to_csv('customers.csv', index=False)
staff.to_csv('staff.csv', index=False)
transaction.to_csv('transaction.csv', index=False)

In [23]:
data.columns

Index(['Date', 'ProductName', 'Quantity', 'UnitPrice', 'StoreLocation',
       'PaymentType', 'PromotionApplied', 'Weather', 'Temperature',
       'StaffPerformanceRating', 'CustomerFeedback', 'DeliveryTime_min',
       'OrderType', 'CustomerName', 'CustomerAddress', 'Customer_PhoneNumber',
       'CustomerEmail', 'Staff_Name', 'Staff_Email', 'DayOfWeek',
       'TotalSales'],
      dtype='object')